In [2]:
import numpy as np
import dask.dataframe as dd
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [3]:
from IPython.display import display

path = '/Users/aaronwang/PycharmProjects/LLMs_n_KGs_for_Education/data/Junyi/junyi/junyi_Exercise_table.csv'

data = pd.read_csv(path, encoding = "utf-8",low_memory=False)
display(data)

,name,live,prerequisites,h_position,v_position,creation_date,seconds_per_fast_problem,pretty_display_name,short_display_name,topic,area
0,parabola_intuition_1,True,recognizing_conic_sections,47,2,2012-10-11 17:55:24.8056 UTC,13.0,?物線直覺 1,?物線直覺1,conic-sections,algebra
1,circles_and_arcs,True,NaN,40,-20,2012-10-11 17:55:33.41014 UTC,27.0,圓與弧,圓與弧,area-perimeter-and-volume,geometry
2,inscribed_angles_3,True,inscribed_angles_2,44,-22,2012-10-11 17:55:44.11836 UTC,5.0,圓周角與圓心角換算 3,圓周角與圓心角換算3,circle-properties,geometry
3,solving_quadratics_by_factoring,True,factoring_polynomials_1,50,-2,2012-10-11 17:54:59.28029 UTC,7.0,因式分解法,因式分解法,quadtratics,algebra
4,graphing_parabolas_1,True,graphing_parabolas_0.5,52,0,2012-10-11 17:55:00.48268 UTC,24.0,畫拋物線 1,畫拋物線1,quadtratics,algebra
...,...,...,...,...,...,...,...,...,...,...,...
832,matrix_basic_distance,True,matrix_basic_percentage,49,16,2014-04-09 04:49:03.7539 UTC,11.0,定義矩陣：城市距離,定義矩陣：城市距離,vectors-matrix,algebra
833,ordering_decimals_new,True,recognize_decimal_percentile_position,13,10,2015-01-19 10:20:24.53248 UTC,8.0,小數大小的排列,小數大小的排列,decimals,arithmetic
834,simplificate_ractional_formula,True,combining_like_terms_2,28,6,2014-07-26 11:08:36.62044 UTC,56.0,分數型算式的化簡,分數型算式的化簡,solving-linear-equations-and-inequalities,algebra
835,geometry_identification_concept,True,congruent_triangles_2,37,-28,2014-08-31 15:52:17.28445 UTC,6.0,證明的意義與應用,證明的意義與應用,congruent-triangles,geometry


In [4]:
data.describe()

,h_position,v_position,seconds_per_fast_problem
count,837.000000,837.000000,837.000000
mean,25.402628,-5.704898,10.782557
std,15.876667,12.721159,8.935352
min,-15.000000,-34.000000,0.000000
25%,15.000000,-17.000000,5.000000
50%,26.000000,-5.000000,8.000000
75%,36.000000,5.000000,13.000000
max,60.000000,19.000000,60.000000


In [6]:
import networkx as nx

### 根据prerequiste来构建知识点图谱，先不去除信息量较低的内容如 xxx_2和xxx_3，其区别只有后边序号

In [10]:
# Create the KG based on KC
G_KC = nx.DiGraph()

# Add edges to the graph
for _, row in data.iterrows():
    G_KC.add_edge(row['prerequisites'], row['name'])

# Remove the node that is np.nan
nan_nodes = [n for n in G_KC if pd.isna(n)]
for node in nan_nodes:
    G_KC.remove_node(node)
name_list = data['name'].unique()
prerequisites_list = data['prerequisites'].unique()

# Add node features
for node in G_KC.nodes():
    print(node)
    # Find the row in df_nodes that matches the node id
    if node in name_list:
        node_data = data[data['name'] == node].iloc[0]
        # Update the node with features from the matching row
        nx.set_node_attributes(G_KC, {node: {'id': node_data['name'], 
                                          'name': node_data['name'], 
                                          'description': node_data['name']}})
    elif node in prerequisites_list:
        node_data = data[data['prerequisites'] == node].iloc[0]
        # Update the node with features from the matching row
        nx.set_node_attributes(G_KC, {node: {'id': node_data['prerequisites'], 
                                          'name': node_data['prerequisites'], 
                                          'description': node_data['prerequisites']}})
    else:
        raise ValueError('Node not in the data!')


recognizing_conic_sections
parabola_intuition_1
circles_and_arcs
inscribed_angles_2
inscribed_angles_3
factoring_polynomials_1
solving_quadratics_by_factoring
graphing_parabolas_0.5
graphing_parabolas_1
domain_of_a_function,range_of_a_function
inverses_of_functions
evaluating_expressions_1
one_step_equations
inequalities_on_a_number_line
one_step_inequalities
multiplying_and_dividing_negative_numbers
number_properties_terminology_1
angles_1
angles_2
recognizing_rays_lines_and_line_segments,angle_rotation
angle_types
addition_2
addition_3
one_step_inequalities,inequalities_and_expressions
compound_inequalities
distributive_property_with_variables
equation_of_a_circle_1
line_relationships,graphing_systems_of_equations
distance_between_point_and_line
distance_formula
midpoint_formula
properties_of_numbers_1
properties_of_numbers_2
quadrilateral_types
properties_of_incenters
measuring_lengths_1
dividing_fractions_0.5
fractional_division_life_problem
volume_box,volume_space
volume_container

In [11]:
def graph_investigation(G_KC):
    print("Number of nodes:", G_KC.number_of_nodes())
    print("Number of edges:", G_KC.number_of_edges())
    
    print("Is the graph directed?", G_KC.is_directed())
    
    # For all nodes
    degrees = dict(G_KC.degree())
    print("Degrees:", degrees)
    
    # For directed graphs, in-degree and out-degree
    if G_KC.is_directed():
        in_degrees = dict(G_KC.in_degree())
        out_degrees = dict(G_KC.out_degree())
        print("In-degrees:", in_degrees)
        print("Out-degrees:", out_degrees)
    
    
    # For undirected graphs
    if not G_KC.is_directed():
        connected_components = list(nx.connected_components(G_KC))
        print("Connected components:", connected_components)
    
    # For directed graphs
    if G_KC.is_directed():
        strongly_connected = list(nx.strongly_connected_components(G_KC))
        weakly_connected = list(nx.weakly_connected_components(G_KC))
        print("Strongly connected components:", strongly_connected)
        print("Number of strongly connected components: ", len(strongly_connected))
        print("Weakly connected components:", weakly_connected)
        print("Number of weakly connected components: ", len(weakly_connected))

In [12]:
graph_investigation(G_KC)

Number of nodes: 1004
Number of edges: 740
Is the graph directed? True
Degrees: {'recognizing_conic_sections': 3, 'parabola_intuition_1': 2, 'circles_and_arcs': 4, 'inscribed_angles_2': 2, 'inscribed_angles_3': 2, 'factoring_polynomials_1': 4, 'solving_quadratics_by_factoring': 3, 'graphing_parabolas_0.5': 2, 'graphing_parabolas_1': 1, 'domain_of_a_function,range_of_a_function': 1, 'inverses_of_functions': 1, 'evaluating_expressions_1': 5, 'one_step_equations': 1, 'inequalities_on_a_number_line': 2, 'one_step_inequalities': 2, 'multiplying_and_dividing_negative_numbers': 5, 'number_properties_terminology_1': 2, 'angles_1': 2, 'angles_2': 7, 'recognizing_rays_lines_and_line_segments,angle_rotation': 1, 'angle_types': 3, 'addition_2': 4, 'addition_3': 1, 'one_step_inequalities,inequalities_and_expressions': 1, 'compound_inequalities': 1, 'distributive_property_with_variables': 0, 'equation_of_a_circle_1': 4, 'line_relationships,graphing_systems_of_equations': 1, 'distance_between_point_a

In [13]:
# Store the course graph
# Store it:
file_name_kp = 'Junyi-Prerequisites.graphml'

nx.write_graphml(G_KC, file_name_kp)